In [44]:
import pandas as pd
import numpy as np
import time as t
from pathlib import Path
from xlwings import view

In [45]:
time_stamp = t.time()

In [46]:
# Source files
file_path_src_pq            = Path("D:/Analytik/3_ФТТ/", "_src_pq.xlsx")
file_path_src_ftt           = Path("D:/Analytik/3_ФТТ/_исх/2023/12/", "фтт.xlsx")
file_path_src_ttzz_squ      = Path("D:/Analytik/3_ФТТ/_исх/", "ттзз_скю.xlsx")

# Manuals dataframe
shimpments_qnt_df           = pd.read_excel(file_path_src_pq, "Количество отгрузок")
departments_df              = pd.read_excel(file_path_src_pq, "Справочник отделов")

# Source dataframe
src_ftt_df                  = pd.read_excel(file_path_src_ftt, header=[0, 1])
src_ttzz_squ                = pd.read_excel(file_path_src_ttzz_squ, header=[0, 1])
# Amounting month
month                       = t.strftime("01.12.2023")

c:\Users\derachic_ai\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [47]:
# Количество отгрузок
shimpments_qnt_df = shimpments_qnt_df.loc[shimpments_qnt_df["Начало месяца"] == month]
shimpments_qnt_df.columns = ["Отдел", "Начало месяца", "Значение", "Атрибут"]

# Преобразование object в date
shimpments_qnt_df['Начало месяца'] = pd.to_datetime(shimpments_qnt_df['Начало месяца'], format='%d.%m.%Y')
shimpments_qnt_df = shimpments_qnt_df.set_index(["Отдел", "Начало месяца"])
shimpments_qnt_df

,,Значение,Атрибут
Отдел,Начало месяца,,
Главмолоко,2023-12-01,26,Количество отгрузок
ФЕРМА,2023-12-01,26,Количество отгрузок
VIP,2023-12-01,26,Количество отгрузок


In [48]:
# Сводную таблицу в плоскую
id_vars = src_ftt_df.columns[0:5].to_list()
value_vars = src_ftt_df.columns[5:].to_list()
df = src_ftt_df.melt(id_vars=id_vars, value_vars=value_vars, value_name='Значение', var_name='Дата').dropna()

# Преобразование object в date
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%y')

# Список полей для переименования
columns = []
for x in df.columns[0:5]:
    columns.append(x[1])    # берем заголовки первого уровня
for x in df.columns[5:]:
    columns.append(x)    

df.columns = columns

df = pd.merge(df, departments_df, on=["Отдел продаж", "Подразделение"])
df["Начало месяца"] = pd.to_datetime(df['Дата']).dt.to_period('M').dt.to_timestamp()

In [49]:
# Продажи, руб.
sales = df.groupby(["Отдел", "Начало месяца"]).sum()
sales["Атрибут"] = "Продажи, руб."
sales


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,6427256.06,"Продажи, руб."
Главмолоко,2023-12-01,39613620.54,"Продажи, руб."
ФЕРМА,2023-12-01,29168559.68,"Продажи, руб."


In [50]:
# Средняя отгрузка
avg_shimpment_df = pd.merge(sales, shimpments_qnt_df, on=["Отдел", "Начало месяца"])
avg_shimpment_df["Значение"] = avg_shimpment_df["Значение_x"] / avg_shimpment_df["Значение_y"]
avg_shimpment_df.drop(columns=["Значение_x", "Значение_y", "Атрибут_x", "Атрибут_y"], inplace=True)
avg_shimpment_df["Атрибут"] = "Средняя отгрузка"
avg_shimpment_df

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,2.472022e+05,Средняя отгрузка
Главмолоко,2023-12-01,1.523601e+06,Средняя отгрузка
ФЕРМА,2023-12-01,1.121868e+06,Средняя отгрузка


In [51]:
# Количество ТРТ
qnt_rtt = df.groupby(["Отдел", "Начало месяца"])["РТТ"].nunique()
qnt_rtt = qnt_rtt.to_frame(name="Значение")
qnt_rtt["Атрибут"] = "Количество ТРТ"
qnt_rtt


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,70,Количество ТРТ
Главмолоко,2023-12-01,2249,Количество ТРТ
ФЕРМА,2023-12-01,2112,Количество ТРТ


In [52]:
# Количество должностей
qnt_posts = df.groupby(["Отдел", "Начало месяца"])["Должность ТА"].nunique()
qnt_posts = qnt_posts.to_frame(name="Значение")
qnt_posts["Атрибут"] = "Количество должностей"
qnt_posts

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,4,Количество должностей
Главмолоко,2023-12-01,26,Количество должностей
ФЕРМА,2023-12-01,20,Количество должностей


In [53]:
# Количество ТА
qnt_ta = df.groupby(["Отдел", "Начало месяца"])["ФИО ТА"].nunique()
qnt_ta = qnt_ta.to_frame(name="Значение")
qnt_ta["Атрибут"] = "Количество ТА"
qnt_ta

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,4,Количество ТА
Главмолоко,2023-12-01,26,Количество ТА
ФЕРМА,2023-12-01,20,Количество ТА


In [54]:
# Количество заказов
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца"])["РТТ"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,999,Количество заказов
Главмолоко,2023-12-01,13311,Количество заказов
ФЕРМА,2023-12-01,10406,Количество заказов


In [55]:
# Среднее кол-во заказов в ТРТ
avg_orders_qnt = pd.merge(qnt_orders, qnt_rtt, on=["Отдел", "Начало месяца"])
avg_orders_qnt["Значение"] = avg_orders_qnt["Значение_x"]/avg_orders_qnt["Значение_y"]
avg_orders_qnt.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_orders_qnt["Атрибут"] = "Среднее кол-во заказов в ТРТ"
avg_orders_qnt

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,14.271429,Среднее кол-во заказов в ТРТ
Главмолоко,2023-12-01,5.918631,Среднее кол-во заказов в ТРТ
ФЕРМА,2023-12-01,4.927083,Среднее кол-во заказов в ТРТ


In [56]:
# Среднее кол-во ТРТ в день на должность
avq_rtt_day_post = qnt_orders.merge(shimpments_qnt_df, on=["Отдел", "Начало месяца"]).merge(qnt_posts, on=["Отдел", "Начало месяца"])
avq_rtt_day_post["Значение"] = avq_rtt_day_post["Значение_x"]/avq_rtt_day_post["Значение_y"]/avq_rtt_day_post["Значение"]
avq_rtt_day_post.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avq_rtt_day_post["Атрибут"] = "Среднее кол-во ТРТ в день на должность"
avq_rtt_day_post

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,9.605769,Среднее кол-во ТРТ в день на должность
Главмолоко,2023-12-01,19.690828,Среднее кол-во ТРТ в день на должность
ФЕРМА,2023-12-01,20.011538,Среднее кол-во ТРТ в день на должность


In [57]:
# Количество ТРТ с грппировкой по ЗЗ
qnt_rtt_by_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "РТТ"])["Дата"].count()
qnt_rtt_by_orders = qnt_rtt_by_orders.to_frame(name="Значение")
qnt_rtt_by_orders["Атрибут"] = "Количество ТРТ с грппировкой по ЗЗ"
qnt_rtt_by_orders["Атрибут1"] = np.where(qnt_rtt_by_orders["Значение"] < 3, "1-2 разовых заказов за месяц",
                                  np.where(qnt_rtt_by_orders["Значение"] < 5, "3-4 разовых заказов за месяц",
                                           np.where(qnt_rtt_by_orders["Значение"] < 8, "5-7 разовых заказов за месяц", 
                                                    np.where(qnt_rtt_by_orders["Значение"] >= 8, "8 и выше заказов за месяц", 0)
                                                    )
                                            )
                                  )
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "Атрибут", "Атрибут1"]).count()
qnt_rtt_by_orders = qnt_rtt_by_orders.reset_index(level=["Атрибут", "Атрибут1"])
qnt_rtt_by_orders

Атрибут  \
Отдел      Начало месяца                                       
VIP        2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
Главмолоко 2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
ФЕРМА      2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-12-01     Количество ТРТ с грппировкой по ЗЗ   

                                              Атрибут1  Значение  
Отдел      Начало месяца                                          
VIP        2023-12-01     1-2 разовых заказов за месяц         7  
           2023-12-01     3-4 разовых заказов за месяц         5  
           2023-12-01     5-7 разовых заказов за месяц        10  
           2023-12-01        8 и выше заказов за месяц        48  
Главмолоко 2023-12-01     1-2 разовых заказов за месяц       424  
           2023-12-01     3-4 разовых заказов за месяц       599  
           2023-12-01     5-7 разовых заказов за месяц       620  
           2023-12-01        8 и выше заказов за месяц       606  
ФЕРМА      2023-12-01     1-2 разовых заказов за месяц       612  
           2023-12-01     3-4 разовых заказов за месяц       638  
           2023-12-01     5-7 разовых заказов за месяц       461  
           2023-12-01        8 и выше заказов за месяц       401

In [58]:
# Средний заказ, руб.
avg_order = sales.merge(qnt_orders, on=["Отдел", "Начало месяца"])
avg_order["Значение"] = avg_order["Значение_x"]/avg_order["Значение_y"]
avg_order.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_order["Атрибут"] = "Средний заказ, руб."
avg_order

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,6433.689750,"Средний заказ, руб."
Главмолоко,2023-12-01,2976.006351,"Средний заказ, руб."
ФЕРМА,2023-12-01,2803.052055,"Средний заказ, руб."


In [59]:
# Средняя продажа в ТРТ, руб
avg_rtt_sale = sales.merge(qnt_rtt, on=["Отдел", "Начало месяца"])
avg_rtt_sale["Значение"] = avg_rtt_sale["Значение_x"]/avg_rtt_sale["Значение_y"]
avg_rtt_sale.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_rtt_sale["Атрибут"] = "Средняя продажа в ТРТ, руб"
avg_rtt_sale

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-12-01,91817.943714,"Средняя продажа в ТРТ, руб"
Главмолоко,2023-12-01,17613.881965,"Средняя продажа в ТРТ, руб"
ФЕРМА,2023-12-01,13810.871061,"Средняя продажа в ТРТ, руб"


In [60]:
# Активный ассортимент
id_vars = src_ttzz_squ.columns[:7].to_list()
value_vars = src_ttzz_squ.columns[7:].to_list()
df2 = src_ttzz_squ.melt(id_vars=id_vars, value_vars=value_vars, var_name=["Начало месяца", "Параметр"]).dropna()

# Преобразование object в date
df2['Начало месяца'] = pd.to_datetime(df2["Начало месяца"], format='%Y.%m')

# Список полей для переименования
columns = []
for x in df2.columns[0:7]:
    columns.append(x[1])    # берем заголовки первого уровня
for x in df2.columns[7:]:
    columns.append(x)    

df2.columns = columns
df2 = pd.merge(df2, departments_df, on=["Отдел продаж", "Подразделение"])

active_assort = df2.groupby(["Отдел", "Начало месяца", "Торговая марка"])["ТМЦ"].nunique()
active_assort = active_assort.to_frame("Значение")
active_assort["Атрибут"] = "Активный ассортимент"
active_assort = active_assort.rename_axis(index={"Торговая марка" : "Атрибут1"})
active_assort = active_assort.reset_index(level= "Атрибут1")
active_assort = active_assort[["Значение", "Атрибут", "Атрибут1"]]
active_assort

Значение               Атрибут              Атрибут1
Отдел Начало месяца                                                      
VIP   2023-10-01           16  Активный ассортимент                Без ТМ
      2023-10-01           26  Активный ассортимент             Белавушка
      2023-10-01            2  Активный ассортимент  Белорусский Стандарт
      2023-10-01           24  Активный ассортимент            Главмолоко
      2023-10-01            4  Активный ассортимент           Доброе утро
...                       ...                   ...                   ...
ФЕРМА 2023-12-01            4  Активный ассортимент  Белорусский Стандарт
      2023-12-01            7  Активный ассортимент            Главмолоко
      2023-12-01           31  Активный ассортимент                 Живая
      2023-12-01            6  Активный ассортимент                 Ретро
      2023-12-01           32  Активный ассортимент                 ФЕРМА

[65 rows x 3 columns]

In [61]:
ftt_df = pd.concat([active_assort
                    , avg_rtt_sale
                    , avg_order
                    , qnt_rtt_by_orders
                    , avq_rtt_day_post
                    , avg_orders_qnt
                    , qnt_orders
                    , qnt_ta
                    , qnt_posts
                    , qnt_rtt
                    , avg_shimpment_df
                    , sales
                    , shimpments_qnt_df
                    ])

# Filter by month
view(ftt_df.loc[[("VIP", pd.to_datetime(month, format='%d.%m.%Y')) 
            , ("Главмолоко", pd.to_datetime(month, format='%d.%m.%Y'))
            , ("ФЕРМА", pd.to_datetime(month, format='%d.%m.%Y'))
            ]]
)


In [62]:
t.time() - time_stamp 

4.237006902694702